In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
import cv2
import os
import json
from tqdm import tqdm
import torch

In [2]:
PROJECT_PATH = '...'
DATA_PATH = '...'
MODEL_PATH = '...'
SUBMIT_PATH = PROJECT_PATH + 'submission_files/'

In [3]:
test_images = sorted([DATA_PATH + x for x in os.listdir(DATA_PATH)])
print(len(test_images))
print(test_images[0], test_images[-1])

44946
/home/mglee/utils/downloads/kw/test/test_0.png /home/mglee/utils/downloads/kw/test/test_9999.png


In [ ]:
from super_gradients.training import models

model = models.get('yolo_nas_s', 
                       num_classes=8,
                       checkpoint_path=MODEL_PATH + 'yolo_nas_s_e50_average_model.pth')
model.to('cuda')

In [5]:
img_id_dict = dict()

with open(PROJECT_PATH + 'test.json', "r") as f:
    label_coco = json.load(f)

for val in label_coco['images']:
    img_id_dict[val['file_name']] = val['id']

img_id_dict['test_0.png']

0

In [6]:
def quantize_model(model):
    
    # torch.quantization.quantize_dynamic() : 
    # 모델을 동적으로 양자화한다. 동적 양자화는 가중치를 정수로 양자화하지만, 활성화는 추론 중에 동적으로 양자화한다. 
    # model은 양자화할 모델을 나타내고 {torch.nn.Linear} 는 어떤 모델을 양자화 할지를 나타내는 구성 요소인데 여기에서는 선형 계층만 양자화 된다. 
    # dtype=torch.qint8 : 양자화된 데이터의 유형을 나타낸다. (모델의 값을 8 비트 정수로 표현)
    quantized_model = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    return quantized_model # 양자화가 완료된 모델은 'quantized_model'로 반환

In [9]:
model_quantized = quantize_model(model)

In [10]:
import time
start = time.time()
results = model_quantized.predict(test_images, conf=0.75)

print(time.time() - start)

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/super_gradients/training/utils/media/image.py:64: UserWarning: It seems you are using predict() with batch input
  warnings.warn("It seems you are using predict() with batch input")
Predicting Images:   0%|                                                                        | 0/44946 [00:00<?, ?it/s]/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/numpy/lib/arraypad.py:487: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)
[2023-11-09 18:09:50] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
Predicting Images: 100%|███████████████████████████████████████████████████████████| 44946/44946 [04:20

472.7555754184723


In [11]:
def make_submission_yolo_nas(result, test_images, file_name):
  with open(PROJECT_PATH + 'test.json', 'r') as f:
      lab = json.load(f)
    
  # 객체가 탐지된 이미지
  cnt = 0
  for pred in result:
      if len(pred.prediction.bboxes_xyxy) != 0:
          cnt +=1
  print('탐지된 파일 수: ',cnt)

  annot = []

  for img_idx, pred in tqdm(enumerate(result)):
      if len(pred.prediction.bboxes_xyxy) != 0:
        for i in range(len(pred.prediction.bboxes_xyxy)):
              result_dict = dict()
              result_dict['id'] = len(annot)
              result_dict['image_id'] = img_id_dict[test_images[img_idx].split('/')[-1]]
              result_dict['category_id'] = int(pred.prediction.labels[i])

              #bbox
              x = float(pred.prediction.bboxes_xyxy[i][0]) # 여기 있는 x가 x0
              y = float(pred.prediction.bboxes_xyxy[i][1]) # y0
              w = float(pred.prediction.bboxes_xyxy[i][2]) - x # 여기 있는게 xmax니까 x 빼주기
              h = float(pred.prediction.bboxes_xyxy[i][3]) - y # ymax니까 y빼주기

              result_dict['bbox'] = [x,y,w,h]
              result_dict['area'] = w * h
              annot.append(result_dict)
  
  lab['annotations'] = annot
  print('탐지된 객체 수: ', len(lab['annotations']))
  print(lab['annotations'][:3])
  cat_list = [x['category_id'] for x in lab['annotations']]
  display(pd.Series(cat_list).value_counts().sort_values(ascending = False))

  with open(f'{SUBMIT_PATH}{file_name}', "w") as json_file:
    json.dump(lab, json_file, ensure_ascii=False, indent=4)

  return lab['annotations']

In [12]:
ann = make_submission_yolo_nas(results, test_images, 'yolo_nas_s_e50_c75.json')

1    1758
3    1480
7     639
2     457
5     135
0      93
4      83
6      43
dtype: int64

탐지된 파일 수:  4478
탐지된 객체 수:  4688
[{'id': 0, 'image_id': 0, 'category_id': 3, 'bbox': [100.0, 218.25, 357.0, 106.25], 'area': 37931.25}, {'id': 1, 'image_id': 2, 'category_id': 3, 'bbox': [0.25, 217.25, 358.75, 117.25], 'area': 42063.4375}, {'id': 2, 'image_id': 293, 'category_id': 5, 'bbox': [88.75, 256.5, 489.75, 145.5], 'area': 71258.625}]



0it [00:00, ?it/s]
44946it [00:00, 994892.41it/s]


In [25]:
with open(f'{SUBMIT_PATH}yolo_nas_s_e50_c75.json', "r") as json_file:
    test_lab = json.load(json_file)

print(len(test_lab))
display(test_lab['annotations'][-3:])

[{'id': 4685,
  'image_id': 289,
  'category_id': 5,
  'bbox': [241.375, 205.0, 398.625, 151.25],
  'area': 60292.03125},
 {'id': 4686,
  'image_id': 3226,
  'category_id': 3,
  'bbox': [-0.875, 212.375, 353.125, 228.875],
  'area': 80821.484375},
 {'id': 4687,
  'image_id': 3227,
  'category_id': 3,
  'bbox': [2.25, 251.75, 537.75, 208.25],
  'area': 111986.4375}]

3
